# Preprocesado de datasets
En este cuaderno, se pasan los distintos dataset para corregirlos al formato correcto. Las imágenes que entran al modelo, deben en formato PNG, por lo que las imágenes de radiografia se convierten a png y si tienen mascara, esta se convierte a png con pixeles con valor 0 o 1, fondo y vertebra respectivamente.
Ademas estas imagenes se organizan en distintas carpetas... no se si especificar las carpetas

In [1]:
#!pip install segmentation-models-pytorch
#!pip install pytorch-lightning
#!pip install matplotlib
#!pip install opencv-python
#!pip install albumentations

In [1]:
import os
import torch
import matplotlib.pyplot as plt
import pytorch_lightning as pl
import segmentation_models_pytorch as smp

from pprint import pprint
from torch.utils.data import DataLoader, Dataset, random_split

import cv2
import numpy as np
from PIL import Image
import albumentations as alb 
from albumentations.pytorch import ToTensorV2
import random
import pickle as pkl

c:\Users\tania\miniconda3\envs\tf_gpu\lib\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.5' (you have '2.0.3'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
ruta_original = 'dataset_original' # Carpeta donde se encuentran las imagenes originales
ruta_procesado = 'dataset_procesado' # Carpeta donde se guardaran las imagenes procesadas

con_mascara = 'dataset_con_mascara' # Carpeta donde se encuentran las imagenes con mascara
sin_mascara = 'dataset_sin_mascara' # Carpeta donde se encuentran las imagenes sin mascara

imagenes = 'imagenes'
mascaras = 'mascaras'

cervical = 'cervical'
lumbar = 'lumbar'
columna = 'columna_completa'

## Preprocesado del dataset NHANES II
Las imagenes de radiografias se convierten de pkl a png.

Las mascaras tambien se pasan a png pero, los pixeles toman valor binario:
  - 0: fondo
  - 1: vértebra

In [3]:
def pkl_2_png_con_mascara(ruta_entrada, ruta_salida):
  cervical_img = os.path.join(ruta_salida, 'cervical', imagenes)
  cervical_masc = os.path.join(ruta_salida, 'cervical', mascaras)
  lumbar_img = os.path.join(ruta_salida, 'lumbar', imagenes)
  lumbar_masc = os.path.join(ruta_salida, 'lumbar', mascaras)

  rutas_salida = [cervical_img, cervical_masc, lumbar_img, lumbar_masc] 

  for ruta in rutas_salida:
    if not os.path.exists(ruta):
      os.makedirs(ruta)
  
  for nombre_fichero in os.listdir(ruta_entrada):
    if nombre_fichero.endswith('.pkl'):
      ruta_imagen = ''
      ruta_mascara = ''
      if 'cervical' in nombre_fichero.lower().split('_'):
        ruta_imagen = cervical_img
        ruta_mascara = cervical_masc
      else:
        ruta_imagen = lumbar_img
        ruta_mascara = lumbar_masc

      ruta_fichero = os.path.join(ruta_entrada, nombre_fichero)

      with open(ruta_fichero, 'rb') as fichero:
        dato = pkl.load(fichero)

      imagen = dato['image']
      mascara = dato['masks']['vertebrae']

      # Guardamos la imagen
      png_imagen = Image.fromarray(imagen)
      ruta_png_imagen = os.path.join(ruta_imagen, nombre_fichero.replace('_masks_4.pkl', '.png'))
      png_imagen.save(ruta_png_imagen)

      # Guardamos la máscara binaria
      png_mascara = Image.fromarray((mascara).astype(np.uint8)) # Convertimos a uint8 para que se guarde correctamente
      ruta_png_mascara = os.path.join(ruta_mascara, nombre_fichero.replace('_masks_4.pkl', '.png'))
      png_mascara.save(ruta_png_mascara)

In [4]:
# Ruta de los datos originales
dataset = 'NHANESII_Annotations_VertebralOsetophytes/nhanes'
ruta_entrada = os.path.join(ruta_original, dataset)

# Rutas de salida de las imágenes y máscaras
ruta_salida = os.path.join(ruta_procesado, con_mascara)

# Convertimos los ficheros pkl a png
pkl_2_png_con_mascara(ruta_entrada, ruta_salida)

## Preprocesado de dataset Rxs
En este caso, solo se dispone de imagenes de radiografias jpg sin mascara, estas se usaran solo para obtener las predicciones.
El preprocesado que se debe realizar es, cambiar su formato a png y redimencionar las imagenes para que sean multiplos de 32.

In [12]:
def jpg_2_png(ruta_entrada, ruta_salida, es_gris=True):
  if not os.path.exists(ruta_salida):
    os.makedirs(ruta_salida)

  for nombre_fichero in os.listdir(ruta_entrada):
    if nombre_fichero.endswith('.jpg'):
      ruta_fichero = os.path.join(ruta_entrada, nombre_fichero)
      
      if es_gris:
        imagen = cv2.imread(ruta_fichero, cv2.IMREAD_GRAYSCALE)
      else:
        pil_imagen = Image.open(ruta_fichero).convert('L') # Convertimos a escala de grises
        imagen = np.array(pil_imagen).astype(np.uint8)
        imagen = cv2.cvtColor(imagen, cv2.COLOR_GRAY2BGR)

      if imagen is not None:
        png_imagen = Image.fromarray(imagen)
        ruta_imagen = os.path.join(ruta_salida, nombre_fichero.replace('.jpg', '.png'))
        png_imagen.save(ruta_imagen)

In [13]:
dataset = 'Rxs'
ruta_entrada_lumbar = os.path.join(ruta_original, dataset, lumbar)
ruta_salida_lumbar = os.path.join(ruta_procesado, sin_mascara, lumbar, imagenes)
jpg_2_png(ruta_entrada_lumbar, ruta_salida_lumbar)

In [ ]:
# Ruta de los datos originales
dataset = 'Rxs'
ruta_entrada_cervical = os.path.join(ruta_original, dataset, cervical)
ruta_entrada_lumbar = os.path.join(ruta_original, dataset, lumbar)
ruta_entrada_columna = os.path.join(ruta_original, dataset, columna)

# Rutas de salida de las imágenes y máscaras
ruta_salida_cervical = os.path.join(ruta_procesado, sin_mascara, cervical, imagenes)
ruta_salida_lumbar = os.path.join(ruta_procesado, sin_mascara, lumbar, imagenes)
ruta_salida_columna = os.path.join(ruta_procesado, sin_mascara, columna, imagenes)

# Convertimos los ficheros jpg a png
jpg_2_png(ruta_entrada_cervical, ruta_salida_cervical)
jpg_2_png(ruta_entrada_lumbar, ruta_salida_lumbar)
jpg_2_png(ruta_entrada_columna, ruta_salida_columna, False)

[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [1 1 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [2 2 2]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
<PIL.Image.Image image mode=RGB size=2844x6235 at 0x1ADBF5BC4C0>
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [1 1 1]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [2 2 2]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
<PIL.Image.Image image mode=RGB size=2844x6235 at 0x1ADBB40BCD0>
[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0